# Linear Algebra
---

An extremely attractive feature of reversible logic is that it maps very naturally to the use of vectors to represent boolean values (inputs/outputs), and matrices to express operations (gates and circuits). With very little extra work, reversible circuits can be turned into simple linear algebra relations, where multiplying an input vector by a circuit matrix gives us the corresponding output.

## 1. Single-Bit Systems

The first step to transition to a linear algebra representation, is turn our fundamental unit of information (the bit) into a column vector. We do this by assigning to the two possible values the bit can take ($0$ or $1$) the following representations:

$$ 
\vec{\mathit{0}} = \begin{bmatrix} 1 \\ 0 \end{bmatrix}
\qquad \qquad
\vec{\mathit{1}} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
$$

I know. At first glance, this seems silly. Adding a whole extra variable appears to be rather redundant, but the many reasons for doing this will become obvious as we progress throughout this textbook. In particular, we will soon see that this vector representation will be a very convenient way to express the likelihood of finding a bit to be either $0$ or $1$ when we don't have full knowledge of how our circuits are behaving in the presence of, for example, noise. It is also worth noting that we have added an arrow on top of $0$ and $1$ to distinguish the vectors themselves from the values they can take inside the brackets.

An easy way to remember this notation is to think of the column vectors above as the position of a light switch. If a Boolean $0$ represents a light bulb being off, we can associate this with a light switch being flipped up ($1$ in the top space of the vector, $0$ at the bottom). Similarly, a Boolean $1$ would represent the light bulb being on, which happens when the switch if flipped down ($0$ in the top space of the vector, $1$ at the bottom):

<img src="images\01_03_01_light_switches.png" align = "center" width="400"/>

The next step is to figure out how to map logic gates to matrices. As discussed in the previous chapter, in the case of a single bit, the only reversible gate we have is the $\text{X}$ gate, which should change a $\vec{\mathit{0}}$ to a $\vec{\mathit{1}}$, and vice versa:

$$
\vec{\mathit{0}} \xrightarrow{\;\; \text{X} \;\;} \vec{\mathit{1}}
\xrightarrow{\;\; \text{X} \;\;} \vec{\mathit{0}},
$$
which in vector notation is equivalent to:

$$
\begin{bmatrix} 1 \\ 0 \end{bmatrix} \xrightarrow{\;\; \text{X} \;\;} \begin{bmatrix} 0 \\ 1 \end{bmatrix}
\xrightarrow{\;\; \text{X} \;\;} \begin{bmatrix} 1 \\ 0 \end{bmatrix}
$$

The matrix $\text{X}$ that produces this "mapping" is given by:

$$\text{X} = \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix}$$


We can verify this by remembering that, if we have a general vector $\vec{x} = \begin{bmatrix} x_0 \\ x_1 \end{bmatrix}$, and multiply it with a matrix $\text{A} = \begin{bmatrix} a_{00} & a_{01} \\ a_{10} & a_{11} \end{bmatrix}$ to get the vector $\vec{y} = \begin{bmatrix} y_0 \\ y_1 \end{bmatrix},$ we need to perform the following operation:

$$ 
\begin{aligned}
\vec{y} &= A \vec{x}
\\
\\
\begin{bmatrix} y_0 \\ y_1 \end{bmatrix} &= \begin{bmatrix} a_{00} & a_{01} \\ a_{10} & a_{11} \end{bmatrix} \begin{bmatrix} x_0 \\ x_1 \end{bmatrix}
\\
\\
\begin{bmatrix} y_0 \\ y_1 \end{bmatrix} &= \begin{bmatrix} a_{00} x_0 + a_{01}x_1 \\ a_{10} x_0 + a_{11}x_1 \end{bmatrix}
\end{aligned}
$$

Let's now try that for $A = \text{X}$ and $\vec{x} = \vec{\mathit{0}}$:
$$
\begin{aligned}
\vec{y} &= \text{X} \vec{\mathit{0}}
\\
\\
\vec{y} &= \begin{bmatrix} 0 & 1 \\ 1 & 0 \end{bmatrix} \begin{bmatrix} 1 \\ 0 \end{bmatrix} 
\\
\\
\vec{y} &= \begin{bmatrix} 0 \times 1 + 1 \times 0 \\ 1 \times 1 + 0 \times 0 \end{bmatrix} = \begin{bmatrix} 0 \\ 1 \end{bmatrix}
\\
\\
\vec{y} &= \vec{\mathit{1}},
\end{aligned}
$$

which checks out. The same procedure can be followed to show $\vec{\mathit{0}} = \text{X}\vec{\mathit{1}}$, but let's go ahead and actually use NumPy to perform these calculations for us and SymPy to display them nicely!:

In [13]:
import numpy as np
import sympy as sp

In [19]:
# Define Zero vector
zero = np.array([[1],
                 [0]])

print(zero)

[[1]
 [0]]


In [20]:
# We can use SymPy to render our vectors nicer in LaTeX
sp.Matrix(zero)

Matrix([
[1],
[0]])

In [12]:
# Define X matrix
X = np.array([[0,1],
              [1,0]])
sp.Matrix(X)

Matrix([
[0, 1],
[1, 0]])

Now, if you are not accustomed to using Numpy, here's a **VERY** important thing to keep in mind. The multiplication symbol `*` does **NOT** perform matrix multiplication of NumPy arrays; instead, it performs element-wise multiplication (aka, the [Hadamard Product](https://en.wikipedia.org/wiki/Hadamard_product_(matrices))). 

To perform matrix multiplication as defined above, we should use the [`numpy.matmul()`](https://numpy.org/doc/stable/reference/generated/numpy.matmul.html) function. However, we can also use the syntactic-sugar symbol `@`, which is a shortcut for the same operation:

In [14]:
# Multiply Zero vector by X to get One vector:
one = X @ zero
sp.Matrix(one)

Matrix([
[0],
[1]])

In [15]:
# Multiply One vector by X to recover Zero vector:
sp.Matrix(X @ one)

Matrix([
[1],
[0]])

As can be seen, multiplying $\text{X}$ by $\vec{\mathit{1}}$ does indeed produce $\vec{\mathit{0}}$.

Now, at this point, I have to confess that I have been lying, but just a little bit. So far, I have stated that after applying the $\text{X}$ gate once, we can recover our original value by reapplying $\text{X}$ again. And although this is in fact true, it is not "formally" correct. You see, in this textbook I want to prioritize accessibility over formalism in the first few chapters, but always making sure we gradually introduce the necessary concepts. And in this particular case, the correct thing to do when we want to "reverse" the effect of a given gate is to apply the **inverse** of its corresponding matrix.

The reason for this is that, if we have a general matrix $A$ acting on a vector $\vec{x}$, and we want to apply now a matrix $B$, such that we recover $x$, then $B \, A$ must be equal a matrix $I$ that leaves the vector $\vec{x}$ (or any matrix in acts on) unchanged:

$$ 
\begin{aligned}
\vec{x} &= B A \vec{x}
\\
\vec{x} &= I \vec{x} .
\end{aligned}
$$

The matrix $I$ is called the identity matrix. From the above expression we have:
$$
\begin{aligned}
B A &= I
\\
B &= I A^{-1},
\end{aligned}
$$

and since $I$ leaves elements unchanged: 

$$B = A^{-1} .$$

So we have uncovered a very important property of the matrices that represent reversible gates/circuits: they must satisfy invertibility. Going back to the $\text{X}$ matrix, what we find is that it is its own inverse:

$$
X^{-1} = X,
$$

and that is why we can simply say that applying $\text{X}$ a second time reverses its effect. Let's confirm that the relationship above is in fact true by using NumPy's inverse function:

In [21]:
# Find X⁻¹ to see it is equal to X:
X_inv = np.linalg.inv(X)
sp.Matrix(X_inv)

Matrix([
[  0, 1.0],
[1.0,   0]])

An equivalent way to confirm this is simply showing $X X = I$, where the identity matrix is given by:

$$I = \begin{bmatrix} 1 & 0 \\ 0 & 1 \end{bmatrix}$$

In [23]:
# X X⁻¹ = I:
I = X @ X
sp.Matrix(I)

Matrix([
[1, 0],
[0, 1]])

## 2. Multi-Bit Systems

In previous chapters, we 